
### Deploying a Model Trained with Amazon SageMaker


A model is trained using Amazon SageMaker and then directly deployed to an endpoint.

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). In this part of the lab, you will train and test an MNIST model on Amazon SageMaker using MXNet and the Gluon API.

In [11]:
import boto3
import os
import sagemaker
import pandas as pd
import numpy as np

from mxnet import gluon
from mxnet.gluon.utils import download

from sagemaker import get_execution_role
from sagemaker.mxnet import MXNet
from sagemaker.mxnet.model import MXNetModel

from IPython.display import HTML

role = get_execution_role()
print(role)

arn:aws:iam::540188452820:role/service-role/AmazonSageMaker-ExecutionRole-20180130T170108


In [3]:
import sagemaker
sagemaker_session = sagemaker.Session()

#### Download Training and Test Data

In [ ]:
gluon.data.vision.MNIST('./data/train', train=True)
gluon.data.vision.MNIST('./data/test', train=False)

In [4]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')
inputs

's3://sagemaker-us-west-2-540188452820/data/DEMO-mnist'

In [5]:
!cat mnist.py

from __future__ import print_function

import logging
import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn
import numpy as np
import json
import time


logging.basicConfig(level=logging.DEBUG)

# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #


def train(current_host, channel_input_dirs, hyperparameters, hosts, num_gpus):
    # SageMaker passes num_cpus, num_gpus and other args we can use to tailor training to
    # the current container environment, but here we just use simple cpu context.
    ctx = mx.cpu()

    # retrieve the hyperparameters we set in notebook (with some defaults)
    batch_size = hyperparameters.get('batch_size', 100)
    epochs = hyperparameters.get('epochs', 10)
    learning_rate = hyperparameters.get('learning_rate', 0.1)
    momentum = hyperparameters.get('momentum', 0.9)
    log_interval = 

#### Run the Training Script on Amazon SageMaker

The `MXNet` class allows you to run your training function on Amazon SageMaker infrastructure. You need to configure it with your training script, an IAM role, the number of training instances, and the training instance type. In this case, you will run your training job on a single `c4.xlarge` instance.

After you have constructed your `MXNet` object, fit it using the data uploaded to Amazon S3. Amazon SageMaker makes sure your data is available in the local filesystem, so your training script can simply read the data from disk.

**Note:** You can ignore any warnings outputted by the below code cell. 

In [12]:
m = MXNet("mnist.py",
          role=role,
          train_instance_count=1,
          train_instance_type="ml.c4.xlarge",
          hyperparameters={'batch_size': 100,
                         'epochs': 10,
                         'learning_rate': 0.1,
                         'momentum': 0.9,
                         'log_interval': 100},
          framework_version='1.2')
m.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2021-01-27-09-39-48-852


2021-01-27 09:39:49 Starting - Starting the training job...
2021-01-27 09:39:52 Starting - Launching requested ML instances......
2021-01-27 09:41:06 Starting - Preparing the instances for training......
2021-01-27 09:42:17 Downloading - Downloading input data
2021-01-27 09:42:17 Training - Downloading the training image...
2021-01-27 09:42:31 Training - Training image download completed. Training in progress.
2021-01-27 09:42:32,552 INFO - root - running container entrypoint
2021-01-27 09:42:32,552 INFO - root - starting train task
2021-01-27 09:42:32,558 INFO - container_support.training - Training starting
2021-01-27 09:42:34,018 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/src/sa

After training, use the `MXNet` object to build and deploy an `MXNetPredictor` object. This creates an Amazon SageMaker endpoint that you can use to perform inference on JSON-encoded multidimensional arrays.

The `deploy` method does the following, in order:

1. Creates an Amazon SageMaker model by calling the `CreateModel` API. The model that you create in Amazon SageMaker holds information such as location of the model artifacts and the inference code image.
1. Creates an endpoint configuration by calling the `CreateEndpointConfig` API. This configuration holds necessary information including the name of the model (which was created in the preceding step) and the resource configuration (the type and number of Machine Learning compute instances to launch for hosting).
1. Creates the endpoint by calling the `CreateEndpoint` API and specifying the endpoint configuration created in the preceding step. Amazon SageMaker launches Machine Learning compute instances as specified in the endpoint configuration, and deploys the model on them.

Launching an endpoint can take up to 10 minutes to complete

In [13]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2021-01-27-09-39-48-852
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2021-01-27-09-39-48-852


---------------------------------------------------------------------------!

You can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which can then be passed to the MXNet predictor. 

In [14]:
HTML(open("input.html").read())

The predictor runs inference on your input data and returns the predicted digit.

In [19]:
response = predictor.predict(data)
print(int(response))

1


#### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)